In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot

class TwoLayerMLP(nn.Module):
    def __init__(self):
        super(TwoLayerMLP, self).__init__()
        self.layer1 = nn.Linear(2, 2)
        self.output_layer = nn.Linear(2, 1)
    
    def forward(self, x):
        x = torch.sigmoid(self.layer1(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# XOR dataset
# Inputs and corresponding labels
X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# Initialize the network
mlp = TwoLayerMLP()

# Loss Function
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.SGD(mlp.parameters(), lr=0.01)

# Number of epochs (iterations over the dataset)
num_epochs = 5000

# Training loop
for epoch in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    Y_pred = mlp(X)

    # Compute and print loss
    loss = criterion(Y_pred, Y)

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 500 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():
    Y_test = mlp(X)
    print("Predicted probabilities:", Y_test)
    print("Predicted labels:", torch.round(Y_test))

# Example input
example_input = torch.tensor([[0.5, -0.2]], dtype=torch.float32)

# Forward pass
y_pred = mlp(example_input)

# Visualize the network
dot = make_dot(y_pred, params=dict(list(mlp.named_parameters()) + [('input', example_input)]))
dot.render('network_graph', format='png', quiet=True)  # This will save the graph as 'network_graph.png' in the working directory.

Epoch [500/5000], Loss: 0.6945
Epoch [1000/5000], Loss: 0.6932
Epoch [1500/5000], Loss: 0.6931
Epoch [2000/5000], Loss: 0.6931
Epoch [2500/5000], Loss: 0.6930
Epoch [3000/5000], Loss: 0.6930
Epoch [3500/5000], Loss: 0.6929
Epoch [4000/5000], Loss: 0.6928
Epoch [4500/5000], Loss: 0.6928
Epoch [5000/5000], Loss: 0.6927
Predicted probabilities: tensor([[0.4755],
        [0.4906],
        [0.5105],
        [0.5234]])
Predicted labels: tensor([[0.],
        [0.],
        [1.],
        [1.]])


'network_graph.png'

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot

class TwoLayerMLP(nn.Module):
    def __init__(self):
        super(TwoLayerMLP, self).__init__()
        self.layer1 = nn.Linear(2, 2)
        self.output_layer = nn.Linear(2, 1)
    
    def forward(self, x):
        x = torch.sigmoid(self.layer1(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# Initialize the MLP
mlp = TwoLayerMLP()

# Example input
example_input = torch.tensor([[0.5, -0.2]], dtype=torch.float32)

# Forward pass
y_pred = mlp(example_input)

# Visualize the network
dot = make_dot(y_pred, params=dict(list(mlp.named_parameters()) + [('input', example_input)]))
dot.render('network_graph', format='png', quiet=True)  # This will save the graph as 'network_graph.png' in the working directory.


'network_graph.png'

In [11]:
# Example input: a batch of 1 sample with 2 features
example_input = torch.tensor([[1.0, 1.0]])
# Forward pass to get the output
output = mlp(example_input)

print("Output:", output)

Output: tensor([[0.5007]], grad_fn=<SigmoidBackward0>)
